In [ ]:
import pubchempy as pcp 
import rdkit
from pubchempy import *
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
import kora.install.rdkit
import py3Dmol

#Valenz (Wertigkeit, Anzahl möglicher Bindungen) von
#Kohlenstoff: 4
#Wasserstoff: 1
#Sauerstoff: 2

#Datenstruktur für Atom (Python-Implementation):

from typing import Dict, List

#allgemeines Atom mit Attributen
class Atom:
  idCounter: int = 0
  id: int

  # nachbarn: Dict[Atom, int] = {}
  nachbarn = None
  valenz: int
  name: str
  formel: str

  #__init__ funktioniert wie ein Konstruktur
  def __init__(self, valenz: int, name: str, formel: str):
    # Statischer Counter zur Vergabe eindeutiger IDs
    Atom.idCounter += 1
    self.id = Atom.idCounter
    self.nachbarn = {}

    self.formel = formel
    self.valenz = valenz
    self.name = name

  # Überschreiben die Standardimplementation, um Klasse als Schlüsselwert in einem Dictionary nutzbar zu machen
  # zum vergleichen ob es das gleiche Atom ist (self wird mit instanceToCompare verglichen)
  def __eq__(self, instanceToCompare):
    return hasattr(instanceToCompare, 'id') and self.id == instanceToCompare.id

  def __hash__(self):
    return hash(self.id)
  
  def getValenz(self):
    return self.valenz
  
  def addNachbar(self, nachbar, bindungsAnzahl: int):
    self.nachbarn[nachbar] = bindungsAnzahl
    nachbar.nachbarn[self] = bindungsAnzahl

  def removeNachbar(self, nachbar, bindungsAnzahl: int):
    del self.nachbarn[nachbar]
    del nachbar.nachbarn[self]

  def getAnzahlFreierBindungen(self):
    return self.valenz-sum([bindungsAnzahl for nachbar, bindungsAnzahl in self.nachbarn.items()])

#Wassserstoff, Sauerstoff und Kohlenstoff Klassen

class Wasserstoff(Atom):
  #funktioniert wie ein Konstruktur, super greift auf die Elternklasse zu
  def __init__(self):
    super(Wasserstoff, self).__init__(1, "Wasserstoff","H")

class Sauerstoff(Atom):
  #funktioniert wie ein Konstruktur, super greift auf die Elternklasse zu
  def __init__(self):
    super(Sauerstoff, self).__init__(2, "Sauerstoff","O")

class Kohlenstoff(Atom):
  #funktioniert wie ein Konstruktur, super greift auf die Elternklasse zu
  def __init__(self):
    super(Kohlenstoff, self).__init__(4, "Kohlenstoff","C")

#ENDE der Datenstruktur für Atom.


#Algorithmus zur Generierung eines Moleküls, das aus C-, O- und H-Molekülen besteht:

#Eingabe:
#n .. Anzahl der Atome im resultierenden Molekül
#Ausgabe:
#M .. Liste der Atome im resultierenden Molekül mit |M| >= n

#Aus- bzw. Rückgabe:
#M

#Eigentlicher Algorithmus:
#M := leere Menge. {M .. Molekül}
#Erzeuge zufällig ein Kohlenstoff-, Wasserstoff- oder Sauerstoffatom a.
#Füge a zu M hinzu.
#Solange |M| < n:
  #Wähle zufällig ein Kohlenstoff-, Wasserstoff- oder Sauerstoffatom a aus M aus, das noch freie Bindungen aufweist. 
  #Erzeuge zufällig ein Kohlenstoff-, Wasserstoff- oder Sauerstoffatom b.
  #k := ZufallszahlZwischen[ 1 und Minimun(freie Valenz(a), Valenz(b) ].
  #Verknüpfe a mit b über k Bindungen. {Sowohl a als auch b können danach also noch freie Bindungen aufweisen.}
  #Füge b zu M hinzu.
  #Wenn ( |M| < n ) UND ( kein Atom aus M weist mehr freie Bindungen auf ), dann:
    #b := Wähle zufällig ein Atom a aus M, das nur mit genau einem anderen Atom b verbunden ist.
	#k := Anzahl der Bindungen zwischen a und b.
	#Löse ZufallszahlZwischen[1, k] Bindungen zwischen a und b.
	#Wenn zwischen a und b keine Bindungen mehr bestehen, dann:
	  #Enferne a aus M.
	#Ende Wenn.
  #Ende Wenn.
#Ende Solange.    
#// Jetzt haben wir ein Molekül mit der gewünschten Anzahl von Atomen, aber eventuell noch freien Bindungen.   <------
#// Die freien Bindungen müssen wir nun noch "befriedigen".
#// Wir wollen die Anzahl der Atome weder erhöhen noch verringern.
#// Es bestehen zwei Möglichkeiten, die freien Bindungen zu "befriedigen":
#// 1.) Mehrfachbindungen zu den Atomen anlegen, mit denen das Atom mit den freien Bindungen noch verbunden ist, oder
#// 2.) Das Atom mit den freien Bindungen durch ein Atom ersetzen, das genauso viele Bindungen enthält,
#// wie das ursprüngliche Atom bereits "gebundene" Bindungen aufgewiesen hat.
#// Wir verfolgen beide Möglichkeiten nacheinander:
#// Möglichkeit 1 ("Mehrfachbindungsgrad" erhöhen):
#Solange mindestens ein Atom aus M noch freie Bindungen aufweist:
#  a := wähle ein beliebiges Atom aus M, das noch freie Bindungen aufweist.
#  N := ermittle die Menge aller Nachbarn von a, die nicht die maximal mögliche Anzahl an Bindungen mit a aufweisen.
#  Solange a noch freie Bindungen aufweist:
#    b := wähle ein beliebiges Atom aus N.
#	Verknüpfe a und b über das Maximum der zwischen a und b noch offenen Bindungen.
#	Wenn b keine freien Bindungen mehr aufweist:
#        Entferne b aus N.
#    Ende Wenn.	
#  Ende Solange.  
#Ende Solange.
#// Möglichkeit 2 (Atome mit freien Bindungen gegen welche mit passender Bindungsanzahl austauschen ODER einfach mit H-Atomen auffüllen):
#Solange mindestens ein Atom aus M noch freie Bindungen aufweist:
#  a := wähle ein beliebiges Atom aus M, das noch freie Bindungen aufweist.
#  d := (Valenz von a) - (freie Bindungen von a). {Anzahl der benötigten Valenz}
#  Wenn d == Valenz eines verfügbaren Atoms: {Wasserstoff, Sauerstoff, Kohlenstoff}, dann:
#    s := Erzeuge Instanz von Atom mit korrekter Valenz.
#    Entferne a aus M und füge s zu M hinzu.
#    Ersetze alle Verbindungen von a mit s in den Nachbarn von a.
#  Sonst:
#    i := 0.
#    Solange i < d:
#      i+=1.
#      h := Erzeuge Wasserstoffatom.
#      Füge h zu M hinzu.
#      Erzeuge Verbindung zwischen a und h.      
#    Ende Solange.
#  Ende Wenn.
#Ende Solange.
#Return M.
#
#Ende des Algorithmus' zur Generierung eines Moleküls, das aus C-, O- und H-Molekülen besteht
#
#Beispiel einer "Substitutionsstabelle":
#Element | freie Bindungen | Lösung
#--------------------------------------------
#C       |               3 | C ersetzen durch H
#C       |               2 | ersetzen durch O
#C       |               1 | einfach ein H anhängen
#O       |               1 | einfach ein H anhängen
#...

#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------
#----------------------------------------------------------------------------------

import random
n = 0
durchläufe_v2 = 1
durchläufe = 0
while durchläufe < 7:
  n=n+1
  if durchläufe < 6:
    durchläufe_v2 = durchläufe_v2+1
  if durchläufe >= 6:
    durchläufe_v2 = durchläufe_v2*2
  for durchgang in range(durchläufe_v2):
    atoms = (Wasserstoff, Sauerstoff, Kohlenstoff)
    M = []
    Molekül_mit_Namen = []
    Atome_mit_freien_Bindungen = []
    Nachbarn_mit_freien_Bindungen = []
    freieBindungen = True
    n =+ n

    def choose_atom(): 
      i = random.uniform(0,1)
      if i <= 0.5:
        random_atom = atoms[2]
      if i > 0.5 and i <= 0.75:
        random_atom = atoms[1]
      if i > 0.75:
        random_atom = atoms[0]
      new_atom = random_atom()
      return new_atom

    def number_of_nachbarn(dict):
      count = 0
      for key, value in dict.items():
        count +=1
      return count

    def bindungen_erhöhen():
      if a.getAnzahlFreierBindungen == 0:
        Nachbarn_mit_freien_Bindungen.clear()                 #wenn Bindugnen erhöht und a keine freien Bindungen mehr hat aber Nachbarn noch, dann muss array gecleart werden   
      b = random.choice(Nachbarn_mit_freien_Bindungen)
      bindungen = b.nachbarn[a] + min(a.getAnzahlFreierBindungen(),b.getAnzahlFreierBindungen())  #aktuelle Bindungen + minimum an freier Valenz von bei den Atomen (wenn Kohlenstoff 2 frei hat und Sauerstoff 1 dann kann es die Bindung nur um 1 erhöhen)
      b.nachbarn[a] = bindungen   
      a.nachbarn[b] = bindungen   #beide ändern, da sonst in beiden verschiedene Anzahl an Bindungen steht
      if (b.getAnzahlFreierBindungen == 0):                   #wenn Nachbar keine freien Bindungen mehr hat --> aus array entfernen
          Nachbarn_mit_freien_Bindungen.remove(b)
      
      

    def atom_ersetzen(atom, a):
      nachbarn_von_a = []
      neues_atom = atom()

      #print(f'{a} mit {neues_atom} ersetzen')

      for Nachbar in a.nachbarn:
        nachbarn_von_a.append(Nachbar)                              #Nachbarn mit Bindungen von a speichern

      for Nachbar in nachbarn_von_a:                                #extra schleife, weil sonst Dictionary in Schleif geändert wird --> error
        #print(f'Nachbarn von zu ersetzendem Atom: {a.nachbarn}')
        Nachbar.addNachbar(neues_atom, a.nachbarn[Nachbar])
        Nachbar.removeNachbar(a, a.nachbarn[Nachbar])               #alle Nachabrn von a enternen
        #print(f'Nachbarn von zu ersetzendem Atom: {a.nachbarn}')
        #print(f'Nachbarn von neuem Atom: {neues_atom.nachbarn}')

      M.remove(a)
      M.append(neues_atom)
      nachbarn_von_a.clear()

      if len(Atome_mit_freien_Bindungen) > 0:
        Atome_mit_freien_Bindungen.remove(a)

    def wasserstoff_anhängen(a):
      i = 0
      while i < a.getAnzahlFreierBindungen():
        i+=1
        h = atoms[0]
        wasserstoff_atom = h()
        M.append(wasserstoff_atom)
        a.addNachbar(wasserstoff_atom,1)
      if len(Atome_mit_freien_Bindungen) > 0:    #ansonsten versucht es a nochmal zu entfernen, wenn keine Atome mit freien Bindungen mehr da sind --> error             
        Atome_mit_freien_Bindungen.remove(a)     #wenn Nachbar keine freien Bindungen mehr hat --> aus array entfernen


    x = 0
    atom_a = choose_atom()
    M.append(atom_a)

    while len(M) < int(n):
      while (x < 100):                #gehe über gesamtes Molekül+1 (array)
        if x == 99:                         #wenn kein Atom mit freien Bindungen gefunden wurde (dann i > len(M))
            for i in range(len(M)):                     #suche Atom in Molekül mit:
              if (number_of_nachbarn(M[i].nachbarn) == 1):  #nur einen Nachbarn hat
                #print(f'test erster nachbar von {M[i]}: {next(iter(list(M[i].nachbarn.items())[0]))}') #erster nachbar (auch der einzige)
                nachbar_von_M_i = next(iter(list(M[i].nachbarn.items())[0]))
                M[i].nachbarn[nachbar_von_M_i] = M[i].nachbarn[nachbar_von_M_i] - random.randint(1,M[i].nachbarn[nachbar_von_M_i])     #entferne: Bindungen - zw(1 und Anzahl Bindungen)  
                

                if(M[i].nachbarn[nachbar_von_M_i] == 0):   #wenn keine Bindungen mehr exisitieren dann:
                  #print(f'M[i]: {M[i]}')  
                  #print(f'Nachbarn M[i]: {M[i].nachbarn}')
                  #print(f'nachbar_von_M_i]: {nachbar_von_M_i}')
                  #print(f'Nachbarn von nachbar_von_M_i: {nachbar_von_M_i.nachbarn}')                                        
                  M[i].removeNachbar(nachbar_von_M_i,M[i].nachbarn[nachbar_von_M_i])    #Nachbar aus M[i] entfernen                 
                  M.remove(M[i])     #Atom aus M entfernen
                i = 0 #i auf 0 damit das gesamte Molekül wieder von vorn abgesucht wird
                x = 0
                break

        random_atom_a = random.choice(M) 
        #print(f'Random Atom a aus M: {random_atom_a}') 
        x +=1                                       #suche random aus M ein Atom aus
        if random_atom_a.getAnzahlFreierBindungen() > 0: 
          x = 0                       #wenn freie Verbindung dann break aus for loop
          break

        ######if random_atom_a.getAnzahlFreierBindungen() > 0 and i

      random_atom_b = choose_atom()
      #print(f'Random erzeugtes Atom: {random_atom_b}')

      k = random.randint(1, min(random_atom_a.getAnzahlFreierBindungen(),random_atom_b.valenz)) #statt valenz Anzahl freier Bindungen bei a
      #print(f'k: {k}')
      random_atom_a.addNachbar(random_atom_b,k)
      M.append(random_atom_b)

      #print(f'M: {M}')

    #// 1.) Mehrfachbindungen zu den Atomen anlegen, mit denen das Atom mit den freien Bindungen noch verbunden ist, oder
    #// 2.) Das Atom mit den freien Bindungen durch ein Atom ersetzen, das genauso viele Bindungen enthält,
    #// wie das ursprüngliche Atom bereits "gebundene" Bindungen aufgewiesen hat.
    #// Wir verfolgen beide Möglichkeiten nacheinander:
    #// Möglichkeit 1 ("Mehrfachbindungsgrad" erhöhen):
    #Solange mindestens ein Atom aus M noch freie Bindungen aufweist:
    #  a := wähle ein beliebiges Atom aus M, das noch freie Bindungen aufweist.
    #  N := ermittle die Menge aller Nachbarn von a, die nicht die maximal mögliche Anzahl an Bindungen mit a aufweisen.
    #  Solange a noch freie Bindungen aufweist:
    #    b := wähle ein beliebiges Atom aus N.
    #	Verknüpfe a und b über das Maximum der zwischen a und b noch offenen Bindungen.
    #	Wenn b keine freien Bindungen mehr aufweist:
    #        Entferne b aus N.
    #    Ende Wenn.	
    #  Ende Solange.  
    #Ende Solange.
    #// Möglichkeit 2 (Atome mit freien Bindungen gegen welche mit passender Bindungsanzahl austauschen ODER einfach mit H-Atomen auffüllen):
    #Solange mindestens ein Atom aus M noch freie Bindungen aufweist:
    #  a := wähle ein beliebiges Atom aus M, das noch freie Bindungen aufweist.
    #  d := (Valenz von a) - (freie Bindungen von a). {Anzahl der benötigten Valenz}
    #  Wenn d == Valenz eines verfügbaren Atoms: {Wasserstoff, Sauerstoff, Kohlenstoff}, dann:
    #    s := Erzeuge Instanz von Atom mit korrekter Valenz.
    #    Entferne a aus M und füge s zu M hinzu.
    #    Ersetze alle Verbindungen von a mit s in den Nachbarn von a.
    #  Sonst:
    #    i := 0.
    #    Solange i < d:
    #      i+=1.
    #      h := Erzeuge Wasserstoffatom.
    #      Füge h zu M hinzu.
    #      Erzeuge Verbindung zwischen a und h.      
    #    Ende Solange.
    #  Ende Wenn.
    #Ende Solange.
    #Return M.


    ##Möglichkeit 1
    #while freieBindungen == True:
    for y in M:                                         #schauen welches Atom noch freie Verbindungen hat (nachdem M gewünschte Anzahl an Atomen hat)
      if y.getAnzahlFreierBindungen() > 0:              #wenn Atom freie Bindung hat dann in Array einfügen
        Atome_mit_freien_Bindungen.append(y)

    while freieBindungen == True:                       #solange ein Atom mit freien Bindungen existiert
      if len(Atome_mit_freien_Bindungen) > 0:           #wenn es noch ein Atom mit freien Bindungen gibt dann suche ein beliebuges davon aus
        a = random.choice(Atome_mit_freien_Bindungen)
      else:                                             #wenn Array leer ist --> keine Atome mehr mit freien Bindungen 
        break
      
      Nachbarn_mit_freien_Bindungen.clear() 

      for key, value in a.nachbarn.items():             #für jeden Nachbarn von a(Atom mit freien Bindungen)
        if key.getAnzahlFreierBindungen() > 0: 
          Nachbarn_mit_freien_Bindungen.append(key)     #array mit allen Nachbarn die freie Bindungen haben von Atom das freie Bindung hat

      #print(f' Nachbarn von {a} mit freien Bindungen: {Nachbarn_mit_freien_Bindungen}')

      if (len(Nachbarn_mit_freien_Bindungen) > 0 and a.getAnzahlFreierBindungen() > 0):        #wenn Atom Nachbarn mit freien Bindungen hat
        bindungen_erhöhen()
      else:                                             #wenn Atom keine Nachbarn mit freien Bindungen hat, aber selber noch freie Bidnungen hat
        #atome_ersetzen(a) #Durch Atome mit passender Valenz ersetzen oder H Atome auffüllen
        benötigte_valenz = a.valenz - a.getAnzahlFreierBindungen()
        if (benötigte_valenz == 2):
          atom_ersetzen(atoms[1], a)  #a durch Sauerstoff ersetzen
        elif(benötigte_valenz == 1):
          atom_ersetzen(atoms[0], a)   #a durch Wasserstoff ersetzen
        else: 
          wasserstoff_anhängen(a)

    #Notlösung um H anzuhängen:
    for x in M:
      y = 0
      freie_bindungen = x.getAnzahlFreierBindungen()
      if freie_bindungen > 0:
        while y < freie_bindungen:
          wasserstoff_anhängen(x)
          y+=1
          #print('Musste noch ein H drangehangen werden')

      

    #print('------------------------------')
    for i in M:
      #print(f'AtomID: {i.id}')
      #print(f'Name: {i.name} ({i}')
      #print(f'Valenz: {i.valenz}')      #funktioniert, gibt ID von jedem Atom in M aus
      #print(f'Bindungen übrig: {i.getAnzahlFreierBindungen()}')
      #print(f'Nachbarn: {i.nachbarn} ')
      #print('--------------')
      Molekül_mit_Namen.append(i.name)

    atom_formeln = []

    print(f'Molekül: {Molekül_mit_Namen}')
    #print('-------------------------------------------')

    ##Mermaid String
    #print('Mermaid String:')
    H = 0
    O = 0
    C = 0
    for Atom in M:
      if Atom.name == 'Wasserstoff':
        H +=1
      if Atom.name == 'Sauerstoff':
        O +=1
      if Atom.name == 'Kohlenstoff':
        C +=1
      #for Nachbar in Atom.nachbarn:
        #print(f'{Atom.id}(({Atom.name})) ---|{Atom.nachbarn[Nachbar]}| {Nachbar.id}(({Nachbar.name})) ')
    ##Molekül mit einzelnen Formelzeichen
    for Atom in M:
      atom_formeln.append(Atom.formel)
    print(f'Molekül: {atom_formeln}')
    print(f'eventuelle Formel: C{C}H{H}O{O}')

    ##Adjazenzmatrix erstellen
    w, h = len(M), len(M)
    adjazenzmatrix = [[0 for x in range(w)] for y in range(h)] 

    for i in range(len(M)):
      for j in range(len(M)):
        Atom = M[j]
        try:
          adjazenzmatrix[i][j] = M[i].nachbarn[Atom]
        except: 
          adjazenzmatrix[i][j] = 0

    #print(adjazenzmatrix)

    def MolFromGraphs(node_list, adjacency_matrix):

        # create empty editable mol object
        mol = Chem.RWMol()

        # add atoms to mol and keep track of index
        node_to_idx = {}
        for i in range(len(node_list)):
            a = Chem.Atom(node_list[i])
            molIdx = mol.AddAtom(a)
            node_to_idx[i] = molIdx

        # add bonds between adjacent atoms
        for ix, row in enumerate(adjacency_matrix):
            for iy, bond in enumerate(row):

                # only traverse half the matrix
                if iy <= ix:
                    continue

                # add relevant bond type (there are many more of these)
                if bond == 0:
                    continue
                elif bond == 1:
                    bond_type = Chem.rdchem.BondType.SINGLE
                    mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
                elif bond == 2:
                    bond_type = Chem.rdchem.BondType.DOUBLE
                    mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)
                elif bond == 3:
                    bond_type = Chem.rdchem.BondType.TRIPLE
                    mol.AddBond(node_to_idx[ix], node_to_idx[iy], bond_type)

        # Convert RWMol to Mol object
        mol = mol.GetMol()            

        return mol
    smiles = Chem.MolToSmiles(MolFromGraphs(atom_formeln, adjazenzmatrix))
    print(f'SMILES: {smiles}')

    def show(smi, style='stick'):
      mol = Chem.MolFromSmiles(smi)
      mol = Chem.AddHs(mol)
      AllChem.EmbedMolecule(mol)
      AllChem.MMFFOptimizeMolecule(mol, maxIters=200)
      mblock = Chem.MolToMolBlock(mol)
      view = py3Dmol.view(width=600, height=600)
      view.addModel(mblock, 'mol')
      view.setStyle({style:{}})
      view.zoomTo()
      view.show()
      # example

    print(f'3D Ansicht:')
    try:
      show(f'{smiles}')  # or 'P'
    except:
      print('Molekül ist ungültig')

    print(f'2D Ansicht:' )
    Molekül = Chem.MolFromSmiles(Chem.MolToSmiles(MolFromGraphs(atom_formeln, adjazenzmatrix)))
    Molekül

